# FixSigMeta

The goal of this notebook is to explore how `FixSigMeta` avoid problems when `Foo` wants to get signature from its `__init__` using `inspect.signature`. 

In order to understand the problems `FixSigMeta` is fixing, please read official [docs](https://fastcore.fast.ai/meta.html#fixsigmeta) first

At the beginning, I know very little of the source code of `inspect.signature` or in fact `inspect._signature_from_callable` and have no idea how `FixSigMeta` enable `Foo` to overcome the potential problems.

`Fastdebug` library and its `Fastdb.dbprint` enables me to debug any source code and evaluate the expressions you write sitting above the source code and `Fastdb.print` can display source code with comments I add when debugging with `dbprint`.

At the end of the notebook, I hope to have a nice and detailed document on the exploration and have a in-depth understanding of how both `_signature_from_callable` and `FixSigMeta` work.

Here is what I learnt from this notebook about how `FixSigMeta` solve the potential problems 


> As a metaclass, `FixSigMeta` defines its `__new__` to creates a class instance `Foo` with attribute `__signature__` and store the signature of `__init__`. This way `inspect._signature_from_callable` can directly help `Foo` to get signature from `__signature__` instead of going into `__new__`, `__call__` looking for signatures where the potential problems reside.

## Having a better view in Jupyter notebook

In [ ]:
# from IPython.core.display import display, HTML # a depreciated import
from IPython.display import display, HTML 

In [ ]:
display(HTML("<style>.container { width:100% !important; }</style>"))

## Examples to explore

In [ ]:
import inspect

### `Foo` borrows signature from `Foo.__init__`

In [ ]:
class Foo:
    def __init__(self, a, b, c): pass

    @classmethod
    def clsmed(): pass
    
inspect.signature(Foo)

<Signature (a, b, c)>

### `Base.__new__` stops `Foo` borrows signature from `Foo.__init__`

In [ ]:
class Base: # pass
    def __new__(self, **args): pass  # defines a __new__ 

class Foo(Base):
    def __init__(self, d, e, f): pass
    
inspect.signature(Foo) # not a problem for python 3.9+, but is a problem for python 3.7

<Signature (d, e, f)>

### Using old `_signature_from_callable`

#### Explore

In [ ]:
# sigOld="""
# def _signature_from_callable(obj, *,
#                              follow_wrapper_chains=True,
#                              skip_bound_arg=True,
#                              sigcls):

#     "Private helper function to get signature for arbitrary callable objects."

#     if not callable(obj):
#         raise TypeError('{!r} is not a callable object'.format(obj))

#     if isinstance(obj, types.MethodType):
#         # In this case we skip the first parameter of the underlying
#         # function (usually `self` or `cls`).
#         sig = _signature_from_callable(
#             obj.__func__,
#             follow_wrapper_chains=follow_wrapper_chains,
#             skip_bound_arg=skip_bound_arg,
#             sigcls=sigcls)

#         if skip_bound_arg:
#             return _signature_bound_method(sig)
#         else:
#             return sig

#     # Was this function wrapped by a decorator?
#     if follow_wrapper_chains:
#         obj = unwrap(obj, stop=(lambda f: hasattr(f, "__signature__")))
#         if isinstance(obj, types.MethodType):
#             # If the unwrapped object is a *method*, we might want to
#             # skip its first parameter (self).
#             # See test_signature_wrapped_bound_method for details.
#             return _signature_from_callable(
#                 obj,
#                 follow_wrapper_chains=follow_wrapper_chains,
#                 skip_bound_arg=skip_bound_arg,
#                 sigcls=sigcls)

#     try:
#         sig = obj.__signature__
#     except AttributeError:
#         pass
#     else:
#         if sig is not None:
#             if not isinstance(sig, Signature):
#                 raise TypeError(
#                     'unexpected object {!r} in __signature__ '
#                     'attribute'.format(sig))
#             return sig

#     try:
#         partialmethod = obj._partialmethod
#     except AttributeError:
#         pass
#     else:
#         if isinstance(partialmethod, functools.partialmethod):
#             # Unbound partialmethod (see functools.partialmethod)
#             # This means, that we need to calculate the signature
#             # as if it's a regular partial object, but taking into
#             # account that the first positional argument
#             # (usually `self`, or `cls`) will not be passed
#             # automatically (as for boundmethods)

#             wrapped_sig = _signature_from_callable(
#                 partialmethod.func,
#                 follow_wrapper_chains=follow_wrapper_chains,
#                 skip_bound_arg=skip_bound_arg,
#                 sigcls=sigcls)

#             sig = _signature_get_partial(wrapped_sig, partialmethod, (None,))
#             first_wrapped_param = tuple(wrapped_sig.parameters.values())[0]
#             if first_wrapped_param.kind is Parameter.VAR_POSITIONAL:
#                 # First argument of the wrapped callable is `*args`, as in
#                 # `partialmethod(lambda *args)`.
#                 return sig
#             else:
#                 sig_params = tuple(sig.parameters.values())
#                 assert (not sig_params or
#                         first_wrapped_param is not sig_params[0])
#                 new_params = (first_wrapped_param,) + sig_params
#                 return sig.replace(parameters=new_params)

#     if isfunction(obj) or _signature_is_functionlike(obj):
#         # If it's a pure Python function, or an object that is duck type
#         # of a Python function (Cython functions, for instance), then:
#         return _signature_from_function(sigcls, obj)

#     if _signature_is_builtin(obj):
#         return _signature_from_builtin(sigcls, obj,
#                                        skip_bound_arg=skip_bound_arg)

#     if isinstance(obj, functools.partial):
#         wrapped_sig = _signature_from_callable(
#             obj.func,
#             follow_wrapper_chains=follow_wrapper_chains,
#             skip_bound_arg=skip_bound_arg,
#             sigcls=sigcls)
#         return _signature_get_partial(wrapped_sig, obj)

#     sig = None
#     if isinstance(obj, type):
#         # obj is a class or a metaclass

#         # First, let's see if it has an overloaded __call__ defined
#         # in its metaclass
#         call = _signature_get_user_defined_method(type(obj), '__call__')
#         if call is not None:
#             sig = _signature_from_callable(
#                 call,
#                 follow_wrapper_chains=follow_wrapper_chains,
#                 skip_bound_arg=skip_bound_arg,
#                 sigcls=sigcls)
#         else:
#             # Now we check if the 'obj' class has a '__new__' method
#             new = _signature_get_user_defined_method(obj, '__new__')
#             if new is not None:
#                 sig = _signature_from_callable(
#                     new,
#                     follow_wrapper_chains=follow_wrapper_chains,
#                     skip_bound_arg=skip_bound_arg,
#                     sigcls=sigcls)
#             else:
#                 # Finally, we should have at least __init__ implemented
#                 init = _signature_get_user_defined_method(obj, '__init__')
#                 if init is not None:
#                     sig = _signature_from_callable(
#                         init,
#                         follow_wrapper_chains=follow_wrapper_chains,
#                         skip_bound_arg=skip_bound_arg,
#                         sigcls=sigcls)

#         if sig is None:
#             # At this point we know, that `obj` is a class, with no user-
#             # defined '__init__', '__new__', or class-level '__call__'

#             for base in obj.__mro__[:-1]:
#                 # Since '__text_signature__' is implemented as a
#                 # descriptor that extracts text signature from the
#                 # class docstring, if 'obj' is derived from a builtin
#                 # class, its own '__text_signature__' may be 'None'.
#                 # Therefore, we go through the MRO (except the last
#                 # class in there, which is 'object') to find the first
#                 # class with non-empty text signature.
#                 try:
#                     text_sig = base.__text_signature__
#                 except AttributeError:
#                     pass
#                 else:
#                     if text_sig:
#                         # If 'obj' class has a __text_signature__ attribute:
#                         # return a signature based on it
#                         return _signature_fromstr(sigcls, obj, text_sig)

#             # No '__text_signature__' was found for the 'obj' class.
#             # Last option is to check if its '__init__' is
#             # object.__init__ or type.__init__.
#             if type not in obj.__mro__:
#                 # We have a class (not metaclass), but no user-defined
#                 # __init__ or __new__ for it
#                 if (obj.__init__ is object.__init__ and
#                     obj.__new__ is object.__new__):
#                     # Return a signature of 'object' builtin.
#                     return sigcls.from_callable(object)
#                 else:
#                     raise ValueError(
#                         'no signature found for builtin type {!r}'.format(obj))

#     elif not isinstance(obj, _NonUserDefinedCallables):
#         # An object with __call__
#         # We also check that the 'obj' is not an instance of
#         # _WrapperDescriptor or _MethodWrapper to avoid
#         # infinite recursion (and even potential segfault)
#         call = _signature_get_user_defined_method(type(obj), '__call__')
#         if call is not None:
#             try:
#                 sig = _signature_from_callable(
#                     call,
#                     follow_wrapper_chains=follow_wrapper_chains,
#                     skip_bound_arg=skip_bound_arg,
#                     sigcls=sigcls)
#             except ValueError as ex:
#                 msg = 'no signature found for {!r}'.format(obj)
#                 raise ValueError(msg) from ex

#     if sig is not None:
#         # For classes and objects we skip the first parameter of their
#         # __call__, __new__, or __init__ methods
#         if skip_bound_arg:
#             return _signature_bound_method(sig)
#         else:
#             return sig

#     if isinstance(obj, types.BuiltinFunctionType):
#         # Raise a nicer error message for builtins
#         msg = 'no signature found for builtin function {!r}'.format(obj)
#         raise ValueError(msg)

#     raise ValueError('callable {!r} is not supported by signature'.format(obj))
# """

In [ ]:
# safety = inspect.__dict__['_signature_from_callable']

# import ast

# block = ast.parse(sigOld, mode='exec')
# exec(compile(block, '<string>', mode='exec'), globals().update(inspect.__dict__))

# list(locals().keys())

# # _signature_from_callableOld?

# _signature_from_callable?

# inspect._signature_from_callable = _signature_from_callable
# inspect.signature(Foo) # not a problem for python 3.9+, but is a problem for python 3.7

# inspect._signature_from_callable = safety

# inspect.signature(Foo) # not a problem for python 3.9+, but is a problem for python 3.7

#### Refactory

bring in older version of source code in string format to replace the current version of the func

In [ ]:
sigOld="""
def _signature_from_callable(obj, *,
                             follow_wrapper_chains=True,
                             skip_bound_arg=True,
                             sigcls):

    "Private helper function to get signature for arbitrary callable objects."

    if not callable(obj):
        raise TypeError('{!r} is not a callable object'.format(obj))

    if isinstance(obj, types.MethodType):
        # In this case we skip the first parameter of the underlying
        # function (usually `self` or `cls`).
        sig = _signature_from_callable(
            obj.__func__,
            follow_wrapper_chains=follow_wrapper_chains,
            skip_bound_arg=skip_bound_arg,
            sigcls=sigcls)

        if skip_bound_arg:
            return _signature_bound_method(sig)
        else:
            return sig

    # Was this function wrapped by a decorator?
    if follow_wrapper_chains:
        obj = unwrap(obj, stop=(lambda f: hasattr(f, "__signature__")))
        if isinstance(obj, types.MethodType):
            # If the unwrapped object is a *method*, we might want to
            # skip its first parameter (self).
            # See test_signature_wrapped_bound_method for details.
            return _signature_from_callable(
                obj,
                follow_wrapper_chains=follow_wrapper_chains,
                skip_bound_arg=skip_bound_arg,
                sigcls=sigcls)

    try:
        sig = obj.__signature__
    except AttributeError:
        pass
    else:
        if sig is not None:
            if not isinstance(sig, Signature):
                raise TypeError(
                    'unexpected object {!r} in __signature__ '
                    'attribute'.format(sig))
            return sig

    try:
        partialmethod = obj._partialmethod
    except AttributeError:
        pass
    else:
        if isinstance(partialmethod, functools.partialmethod):
            # Unbound partialmethod (see functools.partialmethod)
            # This means, that we need to calculate the signature
            # as if it's a regular partial object, but taking into
            # account that the first positional argument
            # (usually `self`, or `cls`) will not be passed
            # automatically (as for boundmethods)

            wrapped_sig = _signature_from_callable(
                partialmethod.func,
                follow_wrapper_chains=follow_wrapper_chains,
                skip_bound_arg=skip_bound_arg,
                sigcls=sigcls)

            sig = _signature_get_partial(wrapped_sig, partialmethod, (None,))
            first_wrapped_param = tuple(wrapped_sig.parameters.values())[0]
            if first_wrapped_param.kind is Parameter.VAR_POSITIONAL:
                # First argument of the wrapped callable is `*args`, as in
                # `partialmethod(lambda *args)`.
                return sig
            else:
                sig_params = tuple(sig.parameters.values())
                assert (not sig_params or
                        first_wrapped_param is not sig_params[0])
                new_params = (first_wrapped_param,) + sig_params
                return sig.replace(parameters=new_params)

    if isfunction(obj) or _signature_is_functionlike(obj):
        # If it's a pure Python function, or an object that is duck type
        # of a Python function (Cython functions, for instance), then:
        return _signature_from_function(sigcls, obj)

    if _signature_is_builtin(obj):
        return _signature_from_builtin(sigcls, obj,
                                       skip_bound_arg=skip_bound_arg)

    if isinstance(obj, functools.partial):
        wrapped_sig = _signature_from_callable(
            obj.func,
            follow_wrapper_chains=follow_wrapper_chains,
            skip_bound_arg=skip_bound_arg,
            sigcls=sigcls)
        return _signature_get_partial(wrapped_sig, obj)

    sig = None
    if isinstance(obj, type):
        # obj is a class or a metaclass

        # First, let's see if it has an overloaded __call__ defined
        # in its metaclass
        call = _signature_get_user_defined_method(type(obj), '__call__')
        if call is not None:
            sig = _signature_from_callable(
                call,
                follow_wrapper_chains=follow_wrapper_chains,
                skip_bound_arg=skip_bound_arg,
                sigcls=sigcls)
        else:
            # Now we check if the 'obj' class has a '__new__' method
            new = _signature_get_user_defined_method(obj, '__new__')
            if new is not None:
                sig = _signature_from_callable(
                    new,
                    follow_wrapper_chains=follow_wrapper_chains,
                    skip_bound_arg=skip_bound_arg,
                    sigcls=sigcls)
            else:
                # Finally, we should have at least __init__ implemented
                init = _signature_get_user_defined_method(obj, '__init__')
                if init is not None:
                    sig = _signature_from_callable(
                        init,
                        follow_wrapper_chains=follow_wrapper_chains,
                        skip_bound_arg=skip_bound_arg,
                        sigcls=sigcls)

        if sig is None:
            # At this point we know, that `obj` is a class, with no user-
            # defined '__init__', '__new__', or class-level '__call__'

            for base in obj.__mro__[:-1]:
                # Since '__text_signature__' is implemented as a
                # descriptor that extracts text signature from the
                # class docstring, if 'obj' is derived from a builtin
                # class, its own '__text_signature__' may be 'None'.
                # Therefore, we go through the MRO (except the last
                # class in there, which is 'object') to find the first
                # class with non-empty text signature.
                try:
                    text_sig = base.__text_signature__
                except AttributeError:
                    pass
                else:
                    if text_sig:
                        # If 'obj' class has a __text_signature__ attribute:
                        # return a signature based on it
                        return _signature_fromstr(sigcls, obj, text_sig)

            # No '__text_signature__' was found for the 'obj' class.
            # Last option is to check if its '__init__' is
            # object.__init__ or type.__init__.
            if type not in obj.__mro__:
                # We have a class (not metaclass), but no user-defined
                # __init__ or __new__ for it
                if (obj.__init__ is object.__init__ and
                    obj.__new__ is object.__new__):
                    # Return a signature of 'object' builtin.
                    return sigcls.from_callable(object)
                else:
                    raise ValueError(
                        'no signature found for builtin type {!r}'.format(obj))

    elif not isinstance(obj, _NonUserDefinedCallables):
        # An object with __call__
        # We also check that the 'obj' is not an instance of
        # _WrapperDescriptor or _MethodWrapper to avoid
        # infinite recursion (and even potential segfault)
        call = _signature_get_user_defined_method(type(obj), '__call__')
        if call is not None:
            try:
                sig = _signature_from_callable(
                    call,
                    follow_wrapper_chains=follow_wrapper_chains,
                    skip_bound_arg=skip_bound_arg,
                    sigcls=sigcls)
            except ValueError as ex:
                msg = 'no signature found for {!r}'.format(obj)
                raise ValueError(msg) from ex

    if sig is not None:
        # For classes and objects we skip the first parameter of their
        # __call__, __new__, or __init__ methods
        if skip_bound_arg:
            return _signature_bound_method(sig)
        else:
            return sig

    if isinstance(obj, types.BuiltinFunctionType):
        # Raise a nicer error message for builtins
        msg = 'no signature found for builtin function {!r}'.format(obj)
        raise ValueError(msg)

    raise ValueError('callable {!r} is not supported by signature'.format(obj))
"""

In [ ]:
from inspect import _signature_from_callable

In [ ]:
from fastdebug.core import *

In [ ]:
reliveonce(_signature_from_callable, sigOld, db=True)

after exec: list(locals().keys()): ['func', 'oldfunc', 'alive', 'db', 'safety', 'block', '_signature_from_callable']
before update: inspect.getsourcefile(func.__globals__[func.__name__]): /Users/Natsume/mambaforge/lib/python3.9/inspect.py
after update: inspect.getsourcefile(func.__globals__[func.__name__]): <string>


In [ ]:
inspect.signature(Foo) # not a problem for python 3.9+, but is a problem for python 3.7

<Signature (**args)>

In [ ]:
reliveonce(_signature_from_callable, sigOld, alive=False)

In [ ]:
inspect.signature(Foo)

<Signature (d, e, f)>